Как бы сделать бесконечный список из чисел Фибоначчи? Начинаем с `[1, 1]`, каждое следующее число равно сумме двух предыдущих.

```

fib =    1  1  2  3  5  8 13 21


tail fib =     1  2  3  5  8 13 21
fib =          1  1  2  3  5  8 13 21
-------------------------------------
сумма          2  3  5  8 13 21 34        = tail tail fib

```

fib = 1 : 1 : zipWith (+) fib (tail fib)

take 10 fib

# Классы Типов

Бывают в Haskell функции, которые можно применять к разным типам, например, `show` может превратить в строку значения самых разных типов:

In [9]:
show 10
show "abc"
show [10, 20]
show ["abc", "xyz"]
show True

data Human = Student String | Lecturer String
show (Student "Ilya") -- ошибка, для этого типа show не работает

"10"

"\"abc\""

"[10,20]"

"[\"abc\",\"xyz\"]"

"True"

: 

Или функция (+), она работает для числовых типов:

In [14]:
(+) 1 2
(+) 0.1 0.2
(+) "ab" "cd" -- для строк не работает, только для числовых типов

3

0.30000000000000004

: 

Можно ввести класс типов, это описание некоторого набора функций. Далее можно учить эти функции работать с уже существующими типами.

In [19]:
class Show' a where
    show' :: a -> String

Читается так: тип `a` имеет класс Show, если для него работает функция `show'`, которая имеет тип `a -> String`.
После ввода класса функция `show'` не может работать ни с каким типом, мы должны ее учить работать с разными типами.

Давайте для примера научим ее работать с числами:

In [30]:
instance Show' Int where
    show' 0 = ""
    show' n = show' (n `div` 10) ++ case n `mod` 10 of
                                        0 -> "0"
                                        1 -> "1"
                                        2 -> "2"
                                        3 -> "3"
                                        4 -> "4"
                                        5 -> "5"
                                        6 -> "6"
                                        7 -> "7"
                                        8 -> "8"
                                        9 -> "9"


show' (42::Int) -- нужно явно указать, что 42 имеет тип Int

"42"

Научим еще какой-нибудь тип делать show'

In [33]:
instance Show' Bool where
    show' True = "ok"
    show' False = "not ok"
    
show' (2 == 2)
show' (2 > 10)

show (2 == 2)

"ok"

"not ok"

"True"

Теперь давайте научимся делать show для списков. Понятно, что мы не может показывать произвольный список `[a]`, для этого нужно уметь показывать значения типа `a`. Т.е. мы можем показать `[1, 2, 3]`, но не можем показать `[1.1, 2.23, 6.89]`.
Тогда добавляется указание, что тип `a` сам реализует класс `Show'`:

In [37]:
instance Show' [a] where
    show' l = "some list"
    
show' [10, 20, 30]
show' [1.1, 2.2, 3.3]
-- бесполезно

"some list"

"some list"

In [44]:
instance Show' a => Show' [a] where
    show' [] = "[]"
    show' (h:t) = "[" ++ show' h ++ tailWithCommas ++ "]" where
                     tailWithCommas = foldl (\start element -> start ++ "," ++ show' element) "" t
                     
show' [10::Int, 20, 30]   
show' [True, False, True]
show' [[10::Int, 20], [42::Int, 42::Int, 42::Int]]
show' ["abc", "asdf"] -- потому что для String show' не работает

"[10,20,30]"

"[ok,not ok,ok]"

"[[10,20],[42,42,42]]"

: 

Получается, если сделать `instance Show' Char`, то сразу заработает String, потому что это `[Char]`.

Давайте еще пример какого-нибудь класса. Внутри класса можно ввести несколько функций.

In [64]:
class Figure a where
   area :: a -> Double
   
   perimeter :: a -> Double
   
   --sigma :: a -> Double -- area / perimeter^2
   

data BasicFigure = Square Double | Circle Double
data Polygon = Polygon [(Double, Double)]
-- Polygon [(0, 0), (10, 0), (0, 20)] это треугольник с тремя вершинами:

instance Figure BasicFigure where
    area (Square x) = x * x
    area (Circle x) = pi * x * x
    
    perimeter (Square x) = 4 * x
    perimeter (Circle x) = 2 * pi * x
    
area (Square 10)
area (Circle 4.5)
perimeter (Square 10)

Line 10: Use newtype instead of data
Found:
data Polygon = Polygon [(Double, Double)]
Why not:
newtype Polygon = Polygon [(Double, Double)]

100.0

63.61725123519331

40.0

Кстати. Операция `(+)` или `(*)` определена как:

In [60]:
:type (+)

(+) :: forall a. Num a => a -> a -> a

Т.е. она работает только со значениями одинаковых типов. Поэтому нельзя умножить или сложить `Double` и `Int`. Поэтому мы пользуемся вспомогательной функцией `fromIntegral`, которая имеет такой тип, чтобы целое число стало просто числом.

In [51]:
-- 3.3 * (3::Int) - нельзя
3.3 * (fromIntegral (3::Int))

Line 1: Redundant bracket
Found:
3.3 * (fromIntegral (3 :: Int))
Why not:
3.3 * fromIntegral (3 :: Int)

9.899999999999999